In [1]:
import pandas as pd
import warnings 
warnings.filterwarnings(action='ignore')
import numpy as np
import re
import nltk
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
%matplotlib inline

## 데이터 확인

In [2]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
stop

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [3]:
# 파일 불러오기

train = pd.read_csv('./open/train.csv', encoding='utf-8')
test_x = pd.read_csv('./open/test_x.csv', encoding='utf-8')
submission = pd.read_csv('./open/sample_submission.csv', encoding='utf-8')

### train data
- 실제 작가와 작가가 작성한 Text가 라벨링된 데이터프레임

In [4]:
train

,index,text,author
0,0,"He was almost choking. There was so much, so m...",3
1,1,"“Your sister asked for it, I suppose?”",2
2,2,"She was engaged one day as she walked, in per...",1
3,3,"The captain was in the porch, keeping himself ...",4
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3
...,...,...,...
54874,54874,"“Is that you, Mr. Smith?” odin whispered. “I h...",2
54875,54875,"I told my plan to the captain, and between us ...",4
54876,54876,"""Your sincere well-wisher, friend, and sister...",1
54877,54877,“Then you wanted me to lend you money?”,3


In [5]:
train.author.unique(), train.author.value_counts()

(array([3, 2, 1, 4, 0]), 3    15063
 0    13235
 2    11554
 4     7805
 1     7222
 Name: author, dtype: int64)

### text_x
- train data를 학습시켜 어떤 작가가 작성했는지 분석해야할 데이터프레임

In [6]:
test_x

,index,text
0,0,“Not at all. I think she is one of the most ch...
1,1,"""No,"" replied he, with sudden consciousness, ""..."
2,2,As the lady had stated her intention of scream...
3,3,“And then suddenly in the silence I heard a so...
4,4,His conviction remained unchanged. So far as I...
...,...,...
19612,19612,"At the end of another day or two, odin growing..."
19613,19613,"All afternoon we sat together, mostly in silen..."
19614,19614,"odin, having carried his thanks to odin, proc..."
19615,19615,"Soon after this, upon odin's leaving the room,..."


### submission
- 제출 형태
- 가로 index : 작가 명

In [7]:
submission

,index,0,1,2,3,4
0,0,0,0,0,0,0
1,1,0,0,0,0,0
2,2,0,0,0,0,0
3,3,0,0,0,0,0
4,4,0,0,0,0,0
...,...,...,...,...,...,...
19612,19612,0,0,0,0,0
19613,19613,0,0,0,0,0
19614,19614,0,0,0,0,0
19615,19615,0,0,0,0,0


## 전처리

### nltk.stopwords + train_test_split

In [8]:
X = pd.read_csv('open/word.csv')
X.text = X.text.astype('str')
X

,text,author
0,he was almost choking there was so much so muc...,3
1,your sister asked for it i suppose,2
2,she was engaged one day as she walked in perus...,1
3,the captain was in the porch keeping himself c...,4
4,have mercy gentlemen odin flung up his hands ...,3
...,...,...
54874,is that you mr smith odin whispered i hardly ...,2
54875,i told my plan to the captain and between us w...,4
54876,your sincere well wisher friend and sister luc...,1
54877,then you wanted me to lend you money,3


In [9]:
X.text = X.text.str.split(' ')

In [10]:
X

,text,author
0,"[he, was, almost, choking, there, was, so, muc...",3
1,"[your, sister, asked, for, it, i, suppose]",2
2,"[she, was, engaged, one, day, as, she, walked,...",1
3,"[the, captain, was, in, the, porch, keeping, h...",4
4,"[have, mercy, gentlemen, odin, flung, up, his,...",3
...,...,...
54874,"[is, that, you, mr, smith, odin, whispered, , ...",2
54875,"[i, told, my, plan, to, the, captain, and, bet...",4
54876,"[your, sincere, well, wisher, friend, and, sis...",1
54877,"[then, you, wanted, me, to, lend, you, money]",3


In [11]:
X['text'] = X['text'].apply(lambda x: [word for word in x if word not in (stop)])

In [12]:
X.text = X.text.apply(lambda x : (' ').join(x))

In [13]:
X.text

0        almost choking much much wanted strange exclam...
1                                     sister asked suppose
2        engaged one day walked perusing jane last lett...
3        captain porch keeping carefully way treacherou...
4        mercy gentlemen flung hands  write anyway sham...
                               ...                        
54874             smith whispered  hardly dared hope would
54875    told plan captain us settled details accomplis...
54876               sincere well wisher friend sister lucy
54877                                    wanted lend money
54878                               certainly occurred yes
Name: text, Length: 54879, dtype: object

In [14]:
X

,text,author
0,almost choking much much wanted strange exclam...,3
1,sister asked suppose,2
2,engaged one day walked perusing jane last lett...,1
3,captain porch keeping carefully way treacherou...,4
4,mercy gentlemen flung hands write anyway sham...,3
...,...,...
54874,smith whispered hardly dared hope would,2
54875,told plan captain us settled details accomplis...,4
54876,sincere well wisher friend sister lucy,1
54877,wanted lend money,3


In [15]:
X_train, X_test, y_train, y_test = train_test_split(X.text, X.author, test_size=0.2, random_state=13)

### 벡터라이즈

#### countvectorizer

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorization으로 feature extraction 변환 수행. 
cnt_vect = CountVectorizer(stop_words='english')
cnt_vect.fit(X_train)

CountVectorizer(stop_words='english')

In [17]:
X_train_cnt_vect = cnt_vect.transform(X_train)
X_test_cnt_vect = cnt_vect.transform(X_test)
print('X_train의 CountVectorizer Shape:', X_train_cnt_vect.shape, X_test_cnt_vect.shape)

X_train의 CountVectorizer Shape: (43903, 31625) (10976, 31625)


In [18]:
print(cnt_vect.vocabulary_)

{'stepping': 26576, 'room': 23718, 'handing': 12755, 'friend': 11423, 'letter': 16162, 'soon': 25914, 'came': 4027, 'help': 13137, 'diverted': 8342, 'perplexity': 20341, 'answer': 1184, 'resulting': 23295, 'supposed': 27238, 'doubt': 8535, 'impatience': 13969, 'thing': 28015, 'meant': 17316, 'attributing': 1843, 'brothers': 3653, 'wish': 31193, 'dancing': 6961, 'nature': 18424, 'convinced': 6120, 'superior': 27189, 'rest': 23265, 'world': 31329, 'gradually': 12246, 'withdrew': 31218, 'eyes': 10169, 'turned': 28979, 'watching': 30694, 'appeared': 1307, 'silence': 25232, 'light': 16257, 'slowly': 25572, 'wasting': 30683, 'candles': 4064, 'roused': 23773, 'collapse': 5316, 'red': 22656, 'coals': 5207, 'looked': 16534, 'vacantly': 29975, 'concentrating': 5662, 'attention': 1818, 'knitted': 15620, 'fixed': 10771, 'speaking': 26060, 'lapse': 15867, 'dialogue': 7752, 'asked': 1626, 'younger': 31567, 'brother': 3649, 'going': 12131, 'want': 30601, 'shake': 24819, 'hands': 12765, 'bye': 3910, '

In [19]:
len(cnt_vect.vocabulary_)

31625

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# LogisticRegression을 이용하여 학습/예측/평가 수행.
lr_clf = LogisticRegression()
lr_clf.fit(X_train_cnt_vect, y_train)
pred = lr_clf.predict(X_test_cnt_vect)
print('CountVectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

CountVectorized Logistic Regression의 예측 정확도는 0.698


#### tfidfvectorizer

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer

# TF-IDF Vectorization 적용하여 학습 데이터셋과 테스트 데이터 셋 변환.
tfidf_vect = CountVectorizer(stop_words='english')
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

#### Logistic Regression 적용

In [22]:
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

TF-IDF Logistic Regression의 예측 정확도는 0.698


#### Multinomial Naive Bayes 적용

In [23]:
from sklearn.naive_bayes import MultinomialNB
mu_clf = MultinomialNB().fit(X_train_tfidf_vect, y_train)

In [24]:
# 정식으로 pipeline을 만들고

from sklearn.pipeline import Pipeline

text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB())
])

In [25]:
# 학습 후 test accuracy 적용

text_clf.fit(X_train, y_train)

predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)

0.6756559766763849

#### KNN

In [26]:
X_train_tfidf_vect.shape

(43903, 31625)

In [27]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_tfidf_vect, y_train)

KNeighborsClassifier()

In [28]:
from sklearn.metrics import accuracy_score

pred = knn.predict(X_test_tfidf_vect)
print(accuracy_score(y_test, pred))

0.3553206997084548


In [29]:
from sklearn.metrics import classification_report, confusion_matrix

# print(confusion_matrix(y_test, pred))
# print('-'*50)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.36      0.47      0.41      2723
           1       0.29      0.16      0.20      1381
           2       0.30      0.47      0.37      2332
           3       0.44      0.36      0.40      3029
           4       0.39      0.14      0.20      1511

    accuracy                           0.36     10976
   macro avg       0.36      0.32      0.32     10976
weighted avg       0.36      0.36      0.34     10976



#### 여러 모델들 적용

In [30]:
from sklearn.linear_model import LogisticRegression, RidgeClassifier, RidgeClassifierCV, SGDClassifier
from sklearn.ensemble import (AdaBoostClassifier, GradientBoostingClassifier,
                              RandomForestClassifier)
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier


models = []
models.append(('LogisticRegression', LogisticRegression(random_state=13)))
models.append(('MultinomialNB', MultinomialNB()))
models.append(('RandomForestClassifier', RandomForestClassifier(random_state=13, n_jobs=-1)))
models.append(('DecisionTreeClassifier', DecisionTreeClassifier(random_state=13)))
models.append(('AdaBoostClassifier', AdaBoostClassifier(random_state=13)))
models.append(('GradientBoostingClassifier', GradientBoostingClassifier(random_state=13)))
models.append(('LGBMClassifier', LGBMClassifier(random_state=13)))
models.append(('KNeighborsClassifier', KNeighborsClassifier(n_neighbors=5, n_jobs=-1)))
models.append(('LinearSVC', LinearSVC(C=1, loss='hinge', random_state=13)))
models.append(('XgBoost', XGBClassifier(learning_rate=0.1, max_depth=3, random_state=13, n_jobs=-1)))
models.append(('RidgeClassifier', RidgeClassifier(random_state=13)))
models.append(('SGDClassifier', SGDClassifier(random_state=13, loss='modified_huber')))
# models.append(('RidgeClassifierCV', RidgeClassifierCV(cv=3)))

In [31]:
models

[('LogisticRegression', LogisticRegression(random_state=13)),
 ('MultinomialNB', MultinomialNB()),
 ('RandomForestClassifier',
  RandomForestClassifier(n_jobs=-1, random_state=13)),
 ('DecisionTreeClassifier', DecisionTreeClassifier(random_state=13)),
 ('AdaBoostClassifier', AdaBoostClassifier(random_state=13)),
 ('GradientBoostingClassifier', GradientBoostingClassifier(random_state=13)),
 ('LGBMClassifier', LGBMClassifier(random_state=13)),
 ('KNeighborsClassifier', KNeighborsClassifier(n_jobs=-1)),
 ('LinearSVC', LinearSVC(C=1, loss='hinge', random_state=13)),
 ('XgBoost',
  XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
                colsample_bynode=None, colsample_bytree=None, gamma=None,
                gpu_id=None, importance_type='gain', interaction_constraints=None,
                learning_rate=0.1, max_delta_step=None, max_depth=3,
                min_child_weight=None, missing=nan, monotone_constraints=None,
                n_estimators=100, n_jobs=-

In [ ]:
from sklearn.metrics import accuracy_score

train_score = []
test_score = []
names = []

for name, model in models:
    clf = model
    clf.fit(X_train_tfidf_vect, y_train)
    
    train_pred = clf.predict(X_train_tfidf_vect)
    test_pred = clf.predict(X_test_tfidf_vect)
    
    names.append(name)
    train_score.append(accuracy_score(y_train, train_pred))
    test_score.append(accuracy_score(y_test, test_pred))

In [ ]:
result = pd.DataFrame({'model name': names,
                       'train score': train_score,
                       'test score': test_score}) 
result.sort_values(by='train score', ascending=False).reset_index(drop=True)

In [ ]:
result['diff'] = result['train score'] - result['test score']

In [ ]:
result.round(2).sort_values(by='diff', ascending=True)

##### stop words 필터링을 추가하고 ngram을 기본 (1,1)에서 (1,2)로 변경하여 피처 벡터화

In [ ]:
tfidf_vect = CountVectorizer(stop_words='english', ngram_range=(1,2))
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

##### min-df 조정
- min_df, max_df 는 아무런 결과의 차이를 가지고 오지 못했고,
- sublinear_tf도 영향이 없었다.

In [ ]:
tfidf_vect = CountVectorizer(stop_words='english', sublinear_tf = True)
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

In [ ]:
# sublinear_tf : 높은 TF값들에 대해서 스무딩 처리, TF값에 대해 아웃라이어 처리

In [ ]:
tfidf_vect = CountVectorizer(stop_words='english', max_features=200000)
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

In [ ]:
X_train_tfidf_vect.shape

In [ ]:
# max_features = 20000개 일 때 정확도 0.722
X_train_tfidf_vect.shape

In [ ]:
# max_features = 40000개 일 때 정확도 0.723
# max_features = 80000개 일 때 정확도 0.723
# max_faetures = 200000개 일 때 정확도 0.723

#### GridSearchCV로 TfidfVectorizer parameter 조정
- ngram_range=(1, 2) 가 최적

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('clf', OneVsRestClassifier(MultinomialNB(
        fit_prior=True, class_prior=None))),
])
parameters = {
#     'tfidf__max_df': (0.25, 0.5, 0.75),
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'clf__estimator__alpha': (1e-2, 1e-3)
}

grid_search_tune = GridSearchCV(pipeline, parameters, cv=2, n_jobs=2, verbose=3)
grid_search_tune.fit(X_train, y_train)

print("Best parameters set:")
print(grid_search_tune.best_estimator_.steps)

In [ ]:
grid_search_tune.best_score_

##### GridSearchCV로 LogisticRegression C 하이퍼 파라미터 튜닝

In [ ]:
from sklearn.model_selection import GridSearchCV

# 최적 C 값 도출 튜닝 수행. CV는 3 Fold셋으로 설정.
params = { 'C': [0.01, 0.1, 1, 5, 10], 'random_state': [13]}
grid_cv_lr = GridSearchCV(lr_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)
grid_cv_lr.fit(X_train_tfidf_vect, y_train)
print('Logistic Regression best C parameter:', grid_cv_lr.best_params_)

# 최적 C 값으로 학습된 grid_cv로 예측 수행하고 정확도 평가
pred = grid_cv_lr.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

In [ ]:
grid_cv_lr.best_params_

In [ ]:
grid_cv_lr.best_score_

## 모델 하이퍼파라미터 튜닝

### MultinomialNB

In [ ]:
from sklearn.naive_bayes import MultinomialNB

params = {'alpha': [0.01, 0.1, 0.5, 1.0]}
clf = MultinomialNB()
grid_cv = GridSearchCV(clf, param_grid=params, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)
grid_cv.fit(X_train_tfidf_vect, y_train)
print('MultinomialNB best parameters:', grid_cv.best_params_)
print('MultinomialNB best accuracy score:', grid_cv.best_score_)

In [ ]:
clf = MultinomialNB(alpha=0.1)
clf.fit(X_train_tfidf_vect, y_train)
pred = clf.predict(X_test_tfidf_vect)
print('MultinomialNB accuracy score', accuracy_score(y_test, pred))

### LinearSVC

In [ ]:
import sklearn
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

svm_clf = LinearSVC(C=1, loss='hinge', random_state=13)
svm_clf.fit(X_train_tfidf_vect, y_train)

pred = svm_clf.predict(X_test_tfidf_vect)
accuracy_score(y_test, pred)

In [ ]:
print(classification_report(y_test,pred))

In [ ]:
params = {'C':[0.001, 0.01, 1], 'loss':['squared_hinge', 'hinge'], 'penalty':['l1', 'l2'], 'random_state':[13]}
grid_cv = GridSearchCV(LinearSVC(), param_grid=params, refit = True, verbose=2)
grid_cv.fit(X_train_tfidf_vect, y_train)

In [ ]:
print('LinearSVC Best parameters:', grid_cv.best_params_)
print('LinearSVC Best accruacy score:', grid_cv.best_score_)

In [ ]:
svm_clf = LinearSVC(C=1, loss='hinge', penalty='l2', random_state=13)
svm_clf.fit(X_train_tfidf_vect, y_train)

pred = svm_clf.predict(X_test_tfidf_vect)
print('LinearSVC accuracy score:', accuracy_score(y_test, pred)) 

In [ ]:
svm_clf.decision_function(X_test_tfidf_vect) > 0

In [ ]:
# 교차검증

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=13)

In [ ]:
from sklearn.model_selection import cross_validate

cross_validate(svm_clf, X_train_tfidf_vect, y_train, scoring=None, cv=skf, return_train_score=True)

### SGDClassifier

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(random_state=13, loss='modified_huber')
sgd_clf.fit(X_train_tfidf_vect, y_train)

sgd_clf.predict(X_test_tfidf_vect)

In [ ]:
# 교차 검증

from sklearn.model_selection import cross_val_score

cross_val_score(sgd_clf, X_train_tfidf_vect, y_train, cv=5, scoring='accuracy')

In [ ]:
# 오차 행렬

from sklearn.model_selection import cross_val_predict

y_train_pred = cross_val_predict(sgd_clf, X_train_tfidf_vect, y_train, cv=3)

In [ ]:
confusion_matrix(y_train, y_train_pred)

In [ ]:
params = {
    'alpha': [1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3],
    'loss': ['log', 'modified_huber', 'hinge', 'squared_hinge', 'perceptron'],
    'penalty': ['l2', 'elasticnet'],
    'n_jobs': [-1],
    'random_state': [13],
}

grid_cv = GridSearchCV(sgd_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)
grid_cv.fit(X_train_tfidf_vect, y_train)

In [ ]:
print('SGDClassifier best parameters:', grid_cv.best_params_)
print('SGDClassifier best accuracy score:', grid_cv.best_score_)

In [ ]:
sgd_clf = SGDClassifier(alpha=0.0001, loss='modified_huber', penalty='l2', random_state=13, n_jobs=-1)
sgd_clf.fit(X_train_tfidf_vect, y_train)
pred = sgd_clf.predict(X_test_tfidf_vect)
print('SGDClassifier best accuracy score:', accuracy_score(y_test, pred))

### RidgeClassifier

In [ ]:
rd_clf = RidgeClassifier()
params = {
    'alpha': [0.01, 0.1, 0.5, 1.0],
    'normalize': [True, False],
    'max_iter': [100, 300],
    'random_state': [13]
}

grid_cv = GridSearchCV(rd_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)
grid_cv.fit(X_train_tfidf_vect, y_train)
print('RidgeClassifier best parameters:', grid_cv.best_params_)
print('RidgeClassifier best accuracy score:', grid_cv.best_score_)

In [ ]:
rd_clf = RidgeClassifier(alpha=1.0, max_iter=100, normalize='False', random_state=13)
rd_clf.fit(X_train_tfidf_vect, y_train)
pred = rd_clf.predict(X_test_tfidf_vect)
print('RidgeClassifier best accuracy score:', accuracy_score(y_test, pred))

### LogisticRegression

In [ ]:
clf = LogisticRegression()
params = {
    'C': [0.01, 0.1, 0.5, 1.0],
    'max_iter': [100, 200, 500],
}

grid_cv = GridSearchCV(clf, param_grid=params, scoring='accuracy', verbose=1, n_jobs=-1)
grid_cv.fit(X_train_tfidf_vect, y_train)
print('LogisticRegression best parameters:', grid_cv.best_params_)
print('LogisticRegression best accuracy score:', grid_cv.best_score_)

In [ ]:
lr_clf = LogisticRegression(C=1.0, max_iter=200)
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('LogisticRegression best accuracy score:', accuracy_score(y_test, pred))

### LGBMClassifier

In [ ]:
clf = LGBMClassifier()
params = {
    'learning_rate': [0.005, 0.01],
    'n_estimators': [8],
    'num_leaves': [6,8], # large num_leaves helps improve accuracy but might lead to over-fitting
    'boosting_type' : ['dart'], # for better accuracy -> try dart
    'objective' : ['binary'],
    'max_bin':[255, 510], # large max_bin helps improve accuracy but might slow down training progress
    'random_state' : [13],
#     'colsample_bytree' : [0.64, 0.65, 0.66],
#     'subsample' : [0.7,0.75],
    'reg_alpha' : [1,1.2],
    'reg_lambda' : [1,1.2,1.4],
    }

grid_cv = GridSearchCV(clf, param_grid=params, verbose=1, n_jobs=-1)
grid_cv.fit(X_train_tfidf_vect, y_train)
print('LGBMClassifier best parameters:', grid_cv.best_params_)
print('LGBMClassifier best accuracy score:', grid_cv.best_score_)

In [ ]:
lgb_clf = LGBMClassifier(boosting_type='dart', learning_rate=0.01, max_bin=510, n_estimators=8, num_leaves=8,
                        objective='binary', random_state=13, reg_alpha=1, reg_lambda=1)
lgb_clf.fit(X_train_tfidf_vect, y_train)
pred = lgb_clf.predict(X_test_tfidf_vect)
print('LGBMClassifier best accuracy score:', accuracy_score(y_test, pred))

## 전처리

In [ ]:
abc = pd.merge(X_test, y_test, left_index = True, right_index=True)
abc['pred'] = pred

# 예측이 틀린 결과 -> 데이터 프레임 화
error_df = abc[abc['author'] != abc['pred']]

# CountVectorizer로 단어 갯수 카운트
cv = CountVectorizer(stop_words='english')   
cv_fit=cv.fit_transform(error_df['text'])    
word_list = cv.get_feature_names();    
count_list = cv_fit.toarray().sum(axis=0)

di = dict(zip(word_list,count_list))

# 예측이 틀린 결과 중 가장 많이 나온 단어 20개 정렬
worst_df = pd.DataFrame(list(di.items())).sort_values(by=1, ascending=False)[:20]

In [ ]:
worst_df[0].tolist()

In [ ]:
ㅁㅁㅁㅁㅁ

## 모델 검증

In [ ]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=5)
kf = SGDClassifier(random_state=13, loss='modified_huber')

In [ ]:
for train_idx, test_idx in kfold.split(X):
    print(len(train_idx), len(test_idx))

In [ ]:
# 교차 검증 구현하기

kf = KFold(n_splits=2)

print(kf.get_n_splits(X_train_tfidf_vect))
print(kf)
for train_idx, test_idx in kf.split(X_train_tfidf_vect):
    print('--- idx')
    print(train_idx, test_idx)
    print('--- train data')
    print(X_train_tfidf_vect)
    print('--- val data')
    print(X[test_idx])

In [ ]:
from sklearn.model_selection import cross_val_score

skfold = StratifiedKFold(n_splits=5)
cross_val_score(sgd_clf, X_train_tfidf_vect, y_train, cv=skfold, scoring=None)

In [ ]:
cross_val_score(sgd_clf, X_train_tfidf_vect, y_train, cv=5, scoring='accuracy')

In [ ]:
# train score 와 같이

from sklearn.model_selection import cross_validate

cross_validate(sgd_clf, X_train_tfidf_vect, y_train, scoring=None, cv=skfold, return_train_score=True)

In [ ]:
test_x.text

In [ ]:
X_train

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer

# TF-IDF Vectorization 적용하여 학습 데이터셋과 테스트 데이터 셋 변환.
tfidf_vect_test = TfidfVectorizer(stop_words='english', max_features=30000)
tfidf_vect_test.fit(test_x.text)
X_train_tfidf_test = tfidf_vect_test.transform(test_x.text)

In [ ]:
pred

In [ ]:
pred = lr_clf.predict(X_train_tfidf_test)

In [ ]:
pred

In [ ]:
test_x['author'] = pred

In [ ]:
test_x